In [1]:
import pandas as pd
import re
import sqlite3
from collections import Counter
from bs4 import BeautifulSoup
import re
import requests
import numpy as np

In [2]:
conn = sqlite3.connect('movies.db')

In [3]:
pd.read_sql('SELECT * FROM moves LIMIT 10', conn)

,actors,composer,directors,domestic_revenue,franchise,genres,link,prod_budget,producers,rating,release_date,runtime,studio,title,worldwide_revenue,writers,actor_level,director_level
0,"Robert Downey, Jr., Chris Hemsworth, Mark Ruff...",Alan Silvestri,"Joe Russo, Anthony Russo",857409335,1,Action / Adventure,/movies/?id=marvel2019.htm,356000000,Kevin Feige,PG-13,2019-04-26,181.0,Buena Vista,Avengers: Endgame,2794509335,"Christopher Markus, Stephen McFeely",163,8
1,"Donald Glover, James Earl Jones, Beyonce Knowl...",Hans Zimmer,Jon Favreau,430889078,1,Action / Adventure,/movies/?id=lionkingliveaction.htm,None,"Jon Favreau, Jeffrey Silver",PG,2019-07-19,118.0,Buena Vista,The Lion King (2019),1195789078,None,20,4
2,"Brie Larson, Samuel L. Jackson, Ben Mendelsohn...",None,"Anna Boden, Ryan Fleck",426829839,1,Action / Adventure,/movies/?id=marvel2018a.htm,None,Kevin Feige,PG-13,2019-03-08,128.0,Buena Vista,Captain Marvel,1128274794,None,32,8
3,"Tom Hanks, Tim Allen, Jordan Peele, Keanu Reeves",Randy Newman,Josh Cooley,410050743,1,Animation,/movies/?id=pixar2017.htm,None,Jonas Rivera,G,2019-06-21,100.0,Buena Vista,Toy Story 4,959250743,None,16,4
4,"Tom Holland, Jake Gyllenhaal, Marisa Tomei, Jo...",None,None,360328925,1,Action / Adventure,/movies/?id=spidermanhomecoming2.htm,160000000,None,PG-13,2019-07-02,129.0,Sony / Columbia,Spider-Man: Far from Home,1075328925,None,20,0
5,Will Smith,Alan Menken,Guy Ritchie,350369592,1,Fantasy,/movies/?id=disneyfairytale22019.htm,183000000,None,PG,2019-05-24,128.0,Buena Vista,Aladdin (2019),1025569592,None,4,4
6,"Lupita Nyong'o, Elisabeth Moss",None,Jordan Peele,175005930,0,Thriller,/movies/?id=untitledjordanpeele.htm,20000000,"Jordan Peele, Jason Blum",R,2019-03-22,116.0,Universal,Us,254664460,Jordan Peele,6,4
7,"Keanu Reeves, Halle Berry, Laurence Fishburne",Tyler Bates,Chad Stahelski,170165857,1,Action,/movies/?id=johnwick3.htm,None,None,R,2019-05-17,130.0,Lionsgate/Summit,John Wick: Chapter 3 - Parabellum,320051984,None,10,4
8,"Jay Baruchel, America Ferrera, Cate Blanchett,...",None,Dean DeBlois,160799505,1,Animation,/movies/?id=httyd3.htm,129000000,None,PG,2019-02-22,104.0,Universal,How to Train Your Dragon: The Hidden World,519758180,None,23,4
9,"Kevin Hart, Harrison Ford, Tiffany Haddish, La...",Alexandre Desplat,Chris Renaud,155409025,1,Animation,/movies/?id=illumination2018.htm,80000000,"Janet Healy, Christopher Meledandri",PG,2019-06-07,86.0,Universal,The Secret Life of Pets 2,354009025,None,14,4


In [294]:
df = pd.read_sql('''SELECT * FROM movies''', conn)

In [295]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [296]:
df['domestic_revenue'] = df['domestic_revenue'].apply(lambda x: \
                re.findall(r'(\d*)', x.replace('$', '').replace(',', ''))[0])
df['worldwide_revenue'] = df['worldwide_revenue'].apply(lambda x: \
                re.findall(r'(\d*)', x.replace('$', '').replace(',', ''))[0])

In [297]:
df['prod_budget'] = df['prod_budget'].apply(lambda x: int(re.findall(r'\d*',\
        x.replace('$', ''))[0]) * 1000000 if x != '' and 'million' in x else x)
df['prod_budget'] = df['prod_budget'].apply(lambda x: x.replace('$', '').\
                    replace(',', '') if isinstance(x, int) == False else x)

In [298]:
def get_runtime(x):
    if x != '':
        times = re.findall(r'(\d*)\shrs.\s(\d*)\smin.', x)[0]
        hrs = times[0]
        mins = times[1]

        return (int(times[0]) * 60) + int(mins)
    else:
        pass

In [299]:
df['runtime'] = df['runtime'].apply(get_runtime)

In [300]:
df['release_date'] = pd.DatetimeIndex(df['release_date']).date

In [301]:
for col in df.columns:
    df[col] = df[col].apply(lambda x: np.nan if x == '' else x)

In [302]:
# df.to_sql('movies_clean', conn)

In [303]:
df2 = df.dropna()

In [304]:
df3 = df2[['domestic_revenue', 'franchise', 'genres', 
           'prod_budget', 'rating', 'release_date', 'studio', 'worldwide_revenue']].copy()

In [305]:
df3['month'] = df3['release_date'].apply(lambda x: x.month)

In [306]:
df3.drop('release_date', axis=1, inplace=True)

In [307]:
df3['domestic_revenue'] = df3['domestic_revenue'].astype(int)
df3['worldwide_revenue'] = df3['worldwide_revenue'].astype(int)
df3['prod_budget'] = df3['prod_budget'].astype(int)

In [308]:
df3['month'] = df3['month'].astype('O')

In [309]:
df4 = pd.get_dummies(df3.drop('worldwide_revenue', axis=1))

In [366]:
len(df4.columns)

98